# 🧠 OpenAI Vector Store Setup
Dieses Notebook lädt eine Datei hoch, erstellt einen Vector Store, verbindet ihn mit einem Assistant und ermöglicht die semantische Suche mit GPT-4.

In [ ]:
# 📦 Installiere notwendige Pakete
!pip install openai python-dotenv

In [2]:
# 🔐 API-Key einbinden (über .env Datei empfohlen)
from dotenv import load_dotenv
import os
import openai

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# 📁 Datei hochladen
from pathlib import Path

file_path = "../data/Hundeerziehung.docx"  # Passe dies an deine Datei an
file = openai.files.create(file=open(file_path, "rb"), purpose="assistants")
print("✅ Datei hochgeladen:", file.id)

✅ Datei hochgeladen: file-J2HScBefkRU4jN84Qaqm8z


In [4]:
# 🧠 Vector Store erstellen
from openai import OpenAI
client = OpenAI()
vector_store = client.vector_stores.create(name="MeinVektorStore")
print("📌 Vector Store ID:", vector_store.id)

📌 Vector Store ID: vs_67f82fecba008191a634879760df4aaa


In [5]:
# ➕ Datei zum Vector Store hinzufügen

vector_store_file = client.vector_stores.files.create(
  vector_store_id="vs_67f81d72705881918e228b5b84653a26",
  file_id="file-LSaayutqGYLqEmUR54M4Ht"
)

#openai.beta.vector_stores.file_batches.upload_and_poll(
#    vector_store_id=vector_store.id,
#    files=[file.id]
#)
print("✅ Datei im Vector Store gespeichert")

✅ Datei im Vector Store gespeichert


In [6]:
# 🤖 Assistant erstellen
assistant = client.beta.assistants.create(
    name="Hunde-KI",
    instructions="Beantworte Fragen basierend auf dem Dokument.",
    tools=[{"type": "file_search"}],
    tool_resources={
        "file_search": {
            "vector_store_ids": ["vs_67f81d72705881918e228b5b84653a26"]
        }
    },
    model="gpt-4o",
)
print("🧠 Assistant-ID:", assistant.id)

🧠 Assistant-ID: asst_6q9Ceie1Wm1gks6pDozvihFn


In [7]:
# 💬 Thread starten
thread = openai.beta.threads.create()
print("💬 Thread-ID:", thread.id)

💬 Thread-ID: thread_YqMypo2VF2Ocas5JoA4xDuwU


In [8]:
# ❓ Frage stellen und Antwort erhalten
def frage_stellen(frage: str):
    openai.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=frage,
    )
    run = openai.beta.threads.runs.create_and_poll(
        thread_id=thread.id,
        assistant_id=assistant.id,
    )
    messages = openai.beta.threads.messages.list(thread_id=thread.id)
    for msg in reversed(messages.data):
        if msg.role == "assistant":
            return msg.content[0].text.value
    return "Keine Antwort gefunden."

# Beispiel-Frage:
frage_stellen("Warum bellt ein Hund?")


'Hunde bellen aus verschiedenen Gründen. Hier sind einige häufige Ursachen:\n\n1. **Kommunikation:** Hunde bellen, um zu kommunizieren. Sie können andere Hunde, Tiere oder Menschen warnen.\n\n2. **Aufmerksamkeit:** Wenn ein Hund Aufmerksamkeit möchte, sei es für Futter, Wasser, oder Zuwendung, kann er bellen, um dies zu signalisieren.\n\n3. **Warnung oder Alarm:** Hunde bellen oft, um ihre Besitzer vor potentiellen Gefahren zu warnen oder ihr Revier zu verteidigen.\n\n4. **Langeweile oder Einsamkeit:** Ein Hund kann auch bellen, wenn ihm langweilig ist oder er sich einsam fühlt.\n\n5. **Angst oder Stress:** In beängstigenden oder stressigen Situationen kann ein Hund bellen.\n\n6. **Spiel oder Erregung:** Ein Hund kann auch bellen, wenn er sich freut oder aufgeregt ist, zum Beispiel beim Spielen.\n\n7. **Gewohnheit:** Manchmal entsteht Bellen aus Gewohnheit, wenn ein Hund für Bellen in der Vergangenheit belohnt wurde.\n\nEs ist wichtig, das spezifische Bellen eines Hundes im Kontext zu 

👉 Jetzt kannst du oben deine Datei anpassen und beliebige Fragen stellen.